## Work
1. 試比較 save_best_only 與否的差異
2. 請僅存入將 save_weights_only 設定為 True, 並嘗試 reset ipynb 並將模型與權重重新建回並預測 x_test

In [1]:
import os
import keras

# Disable GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
from sklearn.preprocessing import StandardScaler, MinMaxScaler

scaler = StandardScaler()
def preproc_x(x):
    x = x.astype("float32")
    x = x.reshape(x.shape[0],-1)
    x = scaler.fit_transform(x)
    return x

def preproc_y(y):
    y = keras.utils.to_categorical(y, num_classes=10)
    return y      

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
"""
建立你的神經網路
"""
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

def build_mlp(input_dim, layer_output_units, output_units, regularizer=None, dropout_ratio=None, isBN=False):
    model = Sequential()
    for i, untis in enumerate(layer_output_units):  
        if i == 0:
            model.add(Dense(units=untis, 
                            input_dim=input_dim,
                            kernel_initializer='normal',
                            kernel_regularizer=regularizer, 
                            activation='relu'))
            if isBN:
                model.add(BatchNormalization())
        else:
            model.add(Dense(units=untis,
                            kernel_initializer='normal',
                            kernel_regularizer=regularizer, 
                            activation='relu'))
            if isBN:
                model.add(BatchNormalization())

    if dropout_ratio:
        model.add(Dropout(dropout_ratio))
        
    model.add(Dense(units=output_units,
                kernel_initializer='normal', 
                activation='softmax'))
    return model  

In [6]:
"""
訓練模型
"""
def complie_train_model(model, optimizer, epochs, batch_size, callbacks,   x_train, y_train, x_test, y_test):
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer= optimizer)
    model.fit(x_train, y_train, 
          epochs= epochs, 
          batch_size=batch_size, 
          validation_data=(x_test, y_test), 
          callbacks=callbacks, 
          shuffle=True)
    return model

In [7]:
"""
以視覺畫方式檢視訓練過程
"""
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def show_train_history(results):
    for i, result in enumerate(results):
        history = result["model"].history.history
        tag = "({})".format(result["tag"])
        df_acc = pd.DataFrame({"acc"+tag:history["acc"], "val_acc"+tag:history["val_acc"]})
        df_loss = pd.DataFrame({"loss"+tag:history["loss"], "val_loss"+tag:history["val_loss"]})
        if i ==0:
            df_acc_results = df_acc.copy()
            df_loss_results = df_loss.copy()
        else:
            df_acc_results= pd.concat([df_acc_results, df_acc], axis=1)
            df_loss_results= pd.concat([df_loss_results, df_loss], axis=1)    

    fig, ax = plt.subplots(1,2, figsize=(18,5))
    styles = ['bs-','ro-','y^-', 'g--']    
    g1 = df_acc_results.plot.line(style=styles, ax =ax[0])
    g1.set_xlabel("Epoch")
    g1.set_ylabel("Acc") 
    
    g2 = df_loss_results.plot.line(style=styles, ax =ax[1])
    g2.set_xlabel("Epoch")
    g2.set_ylabel("Loss") 
    return df_acc_results, df_loss_results, g1, g2

In [8]:
"""
設定 callbacks
"""

from keras.callbacks import ModelCheckpoint, EarlyStopping

def setup_callbacks(ismodelckpt=False, isearlystop=False, monitor="val_loss", save_only="best", patience=5):
    cbs=list()
    
    if ismodelckpt:
        modelckpt = ModelCheckpoint(filepath="./tmp.h5",
                                    monitor=monitor, 
                                    save_best_only="best" in save_only,
                                    save_weights_only="weights" in save_only)
        cbs.append(modelckpt)
    if isearlystop:
        earlystop = EarlyStopping(monitor=monitor, 
                                  patience=patience, 
                                  verbose=1)
        cbs.append(earlystop)
    return cbs

In [15]:
"""Code Here
設定超參數
"""
EPOCHS = 20
BATCH_SIZE=256
L2_EXP = 1e-3
LEARNING_RATE = 1e-3
MOMENTUM = 0.95
DROPOUT_EXP = 0.25

# model_checkpoint save_best_only= True

In [16]:
# 載入 Callbacks

results = list()

keras.backend.clear_session() # 把舊的 Graph 清掉
regularizer = keras.regularizers.l1_l2(l1=L2_EXP, l2=L2_EXP)
model = build_mlp(input_dim=x_train.shape[1], layer_output_units=[256,128], output_units=10, regularizer=regularizer, dropout_ratio=DROPOUT_EXP, isBN=True)
optimizer =  keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
callbacks = setup_callbacks(ismodelckpt=True, save_only="best")
model = complie_train_model(model=model, optimizer=optimizer,  epochs=EPOCHS , batch_size=BATCH_SIZE, callbacks=callbacks, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)
results.append({"tag": "save_only=best", "model":model})

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 25s 492us/step - loss: 35.2203 - acc: 0.2837 - val_loss: 33.2024 - val_acc: 0.3743
Epoch 2/20
50000/50000 [==============================] - 17s 338us/step - loss: 31.4660 - acc: 0.3873 - val_loss: 29.6984 - val_acc: 0.4190s - loss: 31.5132 -
Epoch 3/20
50000/50000 [==============================] - 18s 355us/step - loss: 28.1110 - acc: 0.4180 - val_loss: 26.4947 - val_acc: 0.4419
Epoch 4/20
50000/50000 [==============================] - 17s 331us/step - loss: 25.0353 - acc: 0.4398 - val_loss: 23.5606 - val_acc: 0.4556
Epoch 5/20
50000/50000 [==============================] - 30s 602us/step - loss: 22.2186 - acc: 0.4572 - val_loss: 20.8805 - val_acc: 0.4663
Epoch 6/20
50000/50000 [==============================] - 22s 431us/step - loss: 19.6521 - acc: 0.4723 - val_loss: 18.4393 - val_acc: 0.4792
Epoch 7/20
50000/50000 [==============================] - 31s 627us/step - loss: 17.3

# Evaluta test data

In [17]:
train_hist= model.evaluate(x_test, y_test)
print(train_hist)

10000/10000 [==============================] - 4s 352us/step
[3.255628981781006, 0.4997]


# Loadback model and evaluta test data

In [18]:
model = keras.models.load_model("./tmp.h5")
loadback_hist = model.evaluate(x_test, y_test)
print(loadback_hist)

10000/10000 [==============================] - 5s 465us/step
[3.255628981781006, 0.4997]


# model_checkpoint save_weights_only= True

In [19]:
# 載入 Callbacks

results = list()

keras.backend.clear_session() # 把舊的 Graph 清掉
regularizer = keras.regularizers.l1_l2(l1=L2_EXP, l2=L2_EXP)
model = build_mlp(input_dim=x_train.shape[1], layer_output_units=[256,128], output_units=10, regularizer=regularizer, dropout_ratio=DROPOUT_EXP, isBN=True)
optimizer =  keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
callbacks = setup_callbacks(ismodelckpt=True, save_only="weights")
model = complie_train_model(model=model, optimizer=optimizer,  epochs=EPOCHS , batch_size=BATCH_SIZE, callbacks=callbacks, x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test)
results.append({"tag": "save_only=weights", "model":model})

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 22s 442us/step - loss: 35.2115 - acc: 0.2862 - val_loss: 33.1903 - val_acc: 0.3819TA: 2s - loss: 35. - ETA: 1s - l
Epoch 2/20
50000/50000 [==============================] - 18s 363us/step - loss: 31.4616 - acc: 0.3864 - val_loss: 29.6886 - val_acc: 0.4195
Epoch 3/20
50000/50000 [==============================] - 19s 372us/step - loss: 28.1017 - acc: 0.4216 - val_loss: 26.4876 - val_acc: 0.4432
Epoch 4/20
50000/50000 [==============================] - 17s 335us/step - loss: 25.0267 - acc: 0.4424 - val_loss: 23.5529 - val_acc: 0.4587
Epoch 5/20
50000/50000 [==============================] - 17s 331us/step - loss: 22.2089 - acc: 0.4602 - val_loss: 20.8730 - val_acc: 0.4700
Epoch 6/20
50000/50000 [==============================] - 18s 362us/step - loss: 19.6365 - acc: 0.4738 - val_loss: 18.4335 - val_acc: 0.4810
Epoch 7/20
50000/50000 [==============================] - 20s 408us/step

# Evaluta test data

In [20]:
train_hist= model.evaluate(x_test, y_test)
print(train_hist)

10000/10000 [==============================] - 6s 622us/step
[3.25037725982666, 0.4977]


# Loadback model and evaluta test data

In [21]:
model.load_weights("./tmp.h5")
loadback_hist = model.evaluate(x_test, y_test)
print(loadback_hist)

10000/10000 [==============================] - 5s 459us/step
[3.25037725982666, 0.4977]
